#### L'objectif principale de ce script est d'effectuer la phase de tests pour tester notre logique ML de notre app Streamlit, sans tester l’UI. 

### 1 - Importer les librairies nécessaires : 


In [ ]:
import unittest
import joblib       
import pandas as pd
import numpy as np


### 2 - Charger les modéles ML a téster

In [23]:
MODEL_PATH = "C:\\Users\\Amatek\\Downloads\\MACHINE LEARNING PROJET S1\\ml_final\\modele\\svm_model.pkl"
SCALER_PATH = "C:\\Users\\Amatek\\Downloads\\MACHINE LEARNING PROJET S1\\ml_final\\modele\\scaler.pkl"
FEATURES_PATH = "C:\\Users\\Amatek\\Downloads\\MACHINE LEARNING PROJET S1\\ml_final\\modele\\feature_names.pkl"
DATA_PATH = r"C:\Users\Amatek\Downloads\MACHINE LEARNING PROJET S1\ml_final\data\dataset_streamlit.csv"

model = joblib.load(MODEL_PATH)
scaler = joblib.load(SCALER_PATH)
feature_names = joblib.load(FEATURES_PATH)
data = pd.read_csv(DATA_PATH, index_col=0)



### 3 - Classe de tests principale



In [ ]:
class TestMLApp(unittest.TestCase):
    pass

###  4 - Test01 : Chargement des objets ML

In [25]:

class TestMLApp(unittest.TestCase):

    def test_model_loaded(self):
        self.assertIsNotNone(model)

    def test_scaler_loaded(self):
        self.assertIsNotNone(scaler)

    def test_feature_names_loaded(self):
        self.assertIsInstance(feature_names, list)
        self.assertGreater(len(feature_names), 0)


Ces tests permettent de vérifier que :
* Le modèle existe .
* Le scaler est chargé . 
* Les features sont bien une liste non vide. 

### 05 -  Tester la cohérence des features

In [26]:
class TestMLApp(unittest.TestCase):

    def test_features_exist_in_data(self):
        for feature in feature_names:
            self.assertIn(feature, data.columns)


Ce test permet d'éviter une erreur qui est classique en prod "feature not found" . 

### 06 - Maintenant on vas tester sur la prédiction sur un cycle valide : 

In [27]:

class TestMLApp(unittest.TestCase):

    def test_prediction_pipeline(self):
        cycle_id = data.index[0]

        X = data.loc[[cycle_id]]
        X = X[feature_names]
        X_scaled = scaler.transform(X)

        pred = model.predict(X_scaled)[0]
        proba = model.predict_proba(X_scaled)[0, 1]

        self.assertIn(pred, [0, 1])
        self.assertGreaterEqual(proba, 0)
        self.assertLessEqual(proba, 1)


L'importance de ces tests est de vérifier que : 

* La prédiction est bien binaire

* La probabilité est entre 0 et 1

### 07 : Tester la géstion des cycles inéxistant

In [29]:
class TestMLApp(unittest.TestCase):

    def test_invalid_cycle(self):
        invalid_cycle = -999

        self.assertNotIn(invalid_cycle, data.index)


Ce test permet de valider la logique de ce bout de code dans le script 'app.py' : 
    
    if cycle_id not in data.index:
    st.error(...)


### 08 : Tester l'absence de NaN après normalisation : 

In [30]:

class TestMLApp(unittest.TestCase):

    def test_no_nan_after_scaling(self):
        X = data.iloc[[0]][feature_names]
        X_scaled = scaler.transform(X)

        self.assertFalse(np.isnan(X_scaled).any())


Ce test est très important pour éviter un crash silencieux du modèle.

### 09 : Exécution des tests : 

In [31]:

suite = unittest.TestLoader().loadTestsFromTestCase(TestMLApp)
unittest.TextTestRunner(verbosity=2).run(suite)


test_no_nan_after_scaling (__main__.TestMLApp.test_no_nan_after_scaling) ... FAIL

FAIL: test_no_nan_after_scaling (__main__.TestMLApp.test_no_nan_after_scaling)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\Amatek\AppData\Local\Temp\ipykernel_9212\1965074479.py", line 7, in test_no_nan_after_scaling
    self.assertFalse(np.isnan(X_scaled).any())
AssertionError: True is not false

----------------------------------------------------------------------
Ran 1 test in 0.012s

FAILED (failures=1)


<unittest.runner.TextTestResult run=1 errors=0 failures=1>

In [32]:
print(data[feature_names].isna().sum())


VS1_std        0
VS1_max        0
PS1_std        0
PS1_max        0
PS1_median     0
PS2_std        0
PS2_max        0
PS2_median     0
PS3_std        0
PS3_max        0
PS3_median     0
PS5_mean       0
PS5_max        0
PS6_mean       0
PS6_max        0
FS1_median     0
FS1_std        0
FS1_max        0
FS2_median     0
FS2_std        0
FS2_max        0
TS1_mean       0
TS1_max        0
TS1_std        0
TS2_mean       0
TS2_max        0
TS2_std        0
TS3_mean       0
TS3_max        0
TS3_std        0
TS4_mean       0
TS4_max        0
TS4_std        0
EPS1_median    0
EPS1_std       0
EPS1_max       0
CE_value       0
CE_diff        0
CP             0
CP_delta       0
SE             0
SE_delta       1
dtype: int64


Ces tests nous ont permis de détecter la présence de valeurs manquantes dans notre jeu de données. Celles-ci risquent de biaiser les résultats, c’est pourquoi il est nécessaire de les traiter avant de poursuivre l’analyse.